In [1]:
import torch

if torch.backends.mps.is_available():
    print("MPS backend is available.")
else:
    print("MPS backend is not available.")

MPS backend is available.


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import json

In [10]:
%cd Documents/GitHub/Hates-speech-detection/notebooks/

/Users/diana/Documents/GitHub/Hates-speech-detection/notebooks


In [9]:
%cd

/Users/diana


In [11]:
def load_data(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)  
    return pd.DataFrame(data)

df = load_data('../embeddings/fast_russian_embeddings.json')

In [3]:
df.head()

,filename,label,embedding
0,252099-79.txt,0,"[-0.012139108963310719, 0.02229919657111168, 0..."
1,842673-37.txt,0,"[-0.027081843465566635, 0.023928195238113403, ..."
2,843784-82.txt,0,"[0.01613425277173519, 0.025259848684072495, -0..."
3,521607-8.txt,0,"[-0.03023727424442768, -0.0032445243559777737,..."
4,406175-82.txt,0,"[0.021884415298700333, 0.014899949543178082, 0..."


In [12]:
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])
X = np.array(df['embedding'].tolist())
y = df['label_encoded'].values

In [13]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [6]:
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
print("Logistic Regression Test Results:")
print(classification_report(y_test, lr.predict(X_test)))

Logistic Regression Test Results:
              precision    recall  f1-score   support

           0       0.62      0.55      0.58      2921
           1       0.45      0.15      0.22      3780
           2       0.77      0.94      0.85     13050

    accuracy                           0.73     19751
   macro avg       0.62      0.55      0.55     19751
weighted avg       0.69      0.73      0.69     19751



In [7]:
lr_balanced = LogisticRegression(max_iter=1000, class_weight='balanced')
lr_balanced.fit(X_train, y_train)
print("Balanced Logistic Regression Test Results:")
print(classification_report(y_test, lr_balanced.predict(X_test)))

Balanced Logistic Regression Test Results:
              precision    recall  f1-score   support

           0       0.50      0.72      0.59      2921
           1       0.35      0.49      0.41      3780
           2       0.89      0.70      0.78     13050

    accuracy                           0.66     19751
   macro avg       0.58      0.64      0.59     19751
weighted avg       0.73      0.66      0.68     19751



In [8]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)
print("Decision Tree Test Results:")
print(classification_report(y_test, dt.predict(X_test)))

Decision Tree Test Results:
              precision    recall  f1-score   support

           0       0.30      0.31      0.31      2921
           1       0.23      0.24      0.24      3780
           2       0.73      0.72      0.73     13050

    accuracy                           0.57     19751
   macro avg       0.42      0.43      0.42     19751
weighted avg       0.57      0.57      0.57     19751



In [9]:
dt_balanced = DecisionTreeClassifier(random_state=42, class_weight='balanced')
dt_balanced.fit(X_train, y_train)
print("Balanced Decision Tree Test Results:")
print(classification_report(y_test, dt_balanced.predict(X_test)))

Balanced Decision Tree Test Results:
              precision    recall  f1-score   support

           0       0.30      0.31      0.30      2921
           1       0.24      0.24      0.24      3780
           2       0.73      0.72      0.73     13050

    accuracy                           0.57     19751
   macro avg       0.42      0.42      0.42     19751
weighted avg       0.57      0.57      0.57     19751



In [50]:
X = np.array(df['embedding'].tolist(), dtype=np.float32)  
y = df['label_encoded'].values

In [51]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [52]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
val_dataset = TensorDataset(torch.tensor(X_val), torch.tensor(y_val))
test_dataset = TensorDataset(torch.tensor(X_test), torch.tensor(y_test))

In [12]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, num_classes):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, num_classes)
        )
    
    def forward(self, x):
        return self.layers(x)

In [12]:
input_size = X_train.shape[1]
num_classes = len(le.classes_)
batch_size = 32
epochs = 20
learning_rate = 0.001
model = NeuralNetwork(input_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [13]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [21]:
for epoch in range(epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    
    model.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            val_loss += criterion(outputs, labels).item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
    
    val_acc = correct / len(val_dataset)
    print(f"Epoch {epoch+1}/{epochs} | Val Accuracy: {val_acc:.4f}")

Epoch 1/20 | Val Accuracy: 0.7221
Epoch 2/20 | Val Accuracy: 0.7310
Epoch 3/20 | Val Accuracy: 0.7343
Epoch 4/20 | Val Accuracy: 0.7363
Epoch 5/20 | Val Accuracy: 0.7343
Epoch 6/20 | Val Accuracy: 0.7311
Epoch 7/20 | Val Accuracy: 0.7321
Epoch 8/20 | Val Accuracy: 0.7379
Epoch 9/20 | Val Accuracy: 0.7389
Epoch 10/20 | Val Accuracy: 0.7422
Epoch 11/20 | Val Accuracy: 0.7370
Epoch 12/20 | Val Accuracy: 0.7387
Epoch 13/20 | Val Accuracy: 0.7405
Epoch 14/20 | Val Accuracy: 0.7406
Epoch 15/20 | Val Accuracy: 0.7393
Epoch 16/20 | Val Accuracy: 0.7415
Epoch 17/20 | Val Accuracy: 0.7362
Epoch 18/20 | Val Accuracy: 0.7355
Epoch 19/20 | Val Accuracy: 0.7404
Epoch 20/20 | Val Accuracy: 0.7376


In [22]:
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        preds = outputs.argmax(dim=1)
        all_preds.extend(preds.numpy())
        all_labels.extend(labels.numpy())

print("\nNeural Network Test Results:")
print(classification_report(all_labels, all_preds))


Neural Network Test Results:
              precision    recall  f1-score   support

           0       0.65      0.55      0.60      2921
           1       0.44      0.27      0.34      3780
           2       0.80      0.92      0.86     13050

    accuracy                           0.74     19751
   macro avg       0.63      0.58      0.60     19751
weighted avg       0.71      0.74      0.72     19751



In [25]:
class_counts = np.bincount(y_train)
class_weights = 1. / class_counts
class_weights = torch.tensor(class_weights, dtype=torch.float32)
criterion_b = nn.CrossEntropyLoss(weight=class_weights)
model_balanced = NeuralNetwork(input_size, num_classes)
optimizer = optim.Adam(model_balanced.parameters(), lr=0.00001)
for epoch in range(epochs):
    model_balanced.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model_balanced(inputs)
        loss = criterion_b(outputs, labels)
        loss.backward()
        optimizer.step()
    
    
    model_balanced.eval()
    val_loss = 0
    correct_balanced = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model_balanced(inputs)
            val_loss += criterion_b(outputs, labels).item()
            preds_balanced = outputs.argmax(dim=1)
            correct_balanced += (preds_balanced == labels).sum().item()
    
    val_acc = correct_balanced / len(val_dataset)
    print(f"Epoch {epoch+1}/{epochs} | Val Accuracy: {val_acc:.4f}")

Epoch 1/20 | Val Accuracy: 0.6595
Epoch 2/20 | Val Accuracy: 0.6788
Epoch 3/20 | Val Accuracy: 0.6892
Epoch 4/20 | Val Accuracy: 0.6656
Epoch 5/20 | Val Accuracy: 0.6452
Epoch 6/20 | Val Accuracy: 0.6373
Epoch 7/20 | Val Accuracy: 0.6382
Epoch 8/20 | Val Accuracy: 0.6346
Epoch 9/20 | Val Accuracy: 0.6363
Epoch 10/20 | Val Accuracy: 0.6421
Epoch 11/20 | Val Accuracy: 0.6382
Epoch 12/20 | Val Accuracy: 0.6369
Epoch 13/20 | Val Accuracy: 0.6393
Epoch 14/20 | Val Accuracy: 0.6413
Epoch 15/20 | Val Accuracy: 0.6428
Epoch 16/20 | Val Accuracy: 0.6366
Epoch 17/20 | Val Accuracy: 0.6408
Epoch 18/20 | Val Accuracy: 0.6357
Epoch 19/20 | Val Accuracy: 0.6409
Epoch 20/20 | Val Accuracy: 0.6392


In [27]:
model_balanced.eval()
all_preds_balanced = []
all_labels = []
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model_balanced(inputs)
        preds_balanced = outputs.argmax(dim=1)
        all_preds_balanced.extend(preds_balanced.numpy())
        all_labels.extend(labels.numpy())

print("\nNeural Network Test Results:")
print(classification_report(all_labels, all_preds_balanced))


Neural Network Test Results:
              precision    recall  f1-score   support

           0       0.44      0.72      0.55      2921
           1       0.28      0.32      0.30      3780
           2       0.87      0.71      0.78     13050

    accuracy                           0.64     19751
   macro avg       0.53      0.58      0.54     19751
weighted avg       0.69      0.64      0.65     19751



In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.unsqueeze(1) 
        out, _ = self.lstm(x)
        out = out[:, -1, :] 
        out = self.fc(out)
        return out

input_size = X_train.shape[1]
num_classes = len(le.classes_)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = LSTMModel(input_size=input_size, 
                 hidden_size=64, 
                 num_layers=1,  
                 output_size=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
epochs = 10
for epoch in range(epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    model.eval()
    correct = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
    
    val_acc = correct / len(val_dataset)
    print(f"Epoch {epoch+1}/{epochs} | Val Accuracy: {val_acc:.4f}")

Epoch 1/10 | Val Accuracy: 0.7402
Epoch 2/10 | Val Accuracy: 0.7403
Epoch 3/10 | Val Accuracy: 0.7411
Epoch 4/10 | Val Accuracy: 0.7378
Epoch 5/10 | Val Accuracy: 0.7423
Epoch 6/10 | Val Accuracy: 0.7395
Epoch 7/10 | Val Accuracy: 0.7370
Epoch 8/10 | Val Accuracy: 0.7418
Epoch 9/10 | Val Accuracy: 0.7424
Epoch 10/10 | Val Accuracy: 0.7433


In [ ]:
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        preds = outputs.argmax(dim=1)
        all_preds.extend(preds.numpy())
        all_labels.extend(labels.numpy())

print("\nNeural Network Test Results:")
print(classification_report(all_labels, all_preds))


Neural Network Test Results:
              precision    recall  f1-score   support

           0       0.63      0.59      0.61      2921
           1       0.47      0.22      0.30      3780
           2       0.80      0.93      0.86     13050

    accuracy                           0.74     19751
   macro avg       0.63      0.58      0.59     19751
weighted avg       0.71      0.74      0.72     19751



In [34]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, WeightedRandomSampler
import time
from sklearn.metrics import classification_report, f1_score
import numpy as np
from collections import Counter

In [53]:
class AdvancedBiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_prob=0.3):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        
        self.input_net = nn.Sequential(
            nn.Linear(input_size, hidden_size * 2),
            nn.GELU(),
            nn.Dropout(dropout_prob),
            nn.LayerNorm(hidden_size * 2),
            
            nn.Linear(hidden_size * 2, hidden_size),
            nn.Mish()
        )
        
       
        self.lstm_layers = nn.ModuleList()
        self.lstm_norms = nn.ModuleList()
        for i in range(num_layers):
            self.lstm_layers.append(nn.LSTM(
                hidden_size,
                hidden_size // 2,
                batch_first=True,
                bidirectional=True,
                dropout=dropout_prob if i < num_layers - 1 else 0
            ))
            self.lstm_norms.append(nn.LayerNorm(hidden_size))
        

        self.attention = nn.MultiheadAttention(
            embed_dim=hidden_size,
            num_heads=4,
            dropout=dropout_prob,
            batch_first=True
        )
        
      
        self.temp_conv = nn.Sequential(
            nn.Conv1d(hidden_size, hidden_size, kernel_size=3, padding=1, groups=4),
            nn.BatchNorm1d(hidden_size),
            nn.SiLU(),
            nn.Dropout(dropout_prob)
        )
       
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size),
            nn.LayerNorm(hidden_size),
            nn.SiLU(),
            nn.Dropout(dropout_prob),
            
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, x):
        if x.dim() == 1:
            x = x.unsqueeze(0)
            
        x = self.input_net(x)
        x = x.unsqueeze(1)
        
        lstm_out = x
        for lstm, norm in zip(self.lstm_layers, self.lstm_norms):
            new_out, _ = lstm(lstm_out)
            lstm_out = norm(lstm_out + new_out)
        
        conv_out = self.temp_conv(lstm_out.transpose(1, 2)).transpose(1, 2)
        
        attn_out, _ = self.attention(conv_out, conv_out, conv_out)
        
        last_out = attn_out[:, -1, :]
        avg_pool = torch.mean(attn_out, dim=1)
        combined = torch.cat([last_out, avg_pool], dim=1)
        
        return self.classifier(combined)

In [54]:
def initialize_model(X_train, num_classes, train_loader, class_weights=None):
    device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
    
    model_config = {
        'input_size': X_train.shape[1],
        'hidden_size': 512,
        'num_layers': 5,
        'output_size': num_classes,
        'dropout_prob': 0.01
    }
    
    model = AdvancedBiLSTM(**model_config).to(device)
    
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=0.0001,
        weight_decay=1e-4
    )
    
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, 
        mode='max',
        factor=0.5,
        patience=2
    )
    
    class_weights = torch.tensor(np.log(class_weights + 1) * 0.5, dtype=torch.float32).to(device)
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    
    return device, model, optimizer, criterion, scheduler


In [55]:
def create_weighted_sampler(y):
    class_counts = Counter(y)
    num_samples = len(y)
    class_weights = {cls: num_samples / count for cls, count in class_counts.items()}
    weights = [class_weights[cls] for cls in y]
    sampler = WeightedRandomSampler(weights, num_samples, replacement=True)
    return sampler

def train_model(model, train_loader, val_loader, optimizer, criterion, device, 
                scheduler=None, epochs=50, clip_value=1.0, patience=5):
    best_val_f1 = 0
    no_improve = 0
    
    for epoch in range(epochs):
        start_time = time.time()
        model.train()
        train_loss = 0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            
            with torch.cuda.amp.autocast(enabled=device.type == 'cuda'):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
            optimizer.step()
            
            train_loss += loss.item() * inputs.size(0)
        
        model.eval()
        val_loss = 0
        all_preds = []
        all_labels = []
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        
        train_loss /= len(train_loader.dataset)
        val_loss /= len(val_loader.dataset)
   
        val_f1 = f1_score(all_labels, all_preds, average='weighted')
        
        if scheduler:
            scheduler.step(val_f1)
        
        epoch_time = time.time() - start_time
        current_lr = optimizer.param_groups[0]['lr']
        print(f'Epoch {epoch+1}/{epochs} | Time: {epoch_time:.2f}s')
        print(f'Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}')
        print(f'Val F1-score: {val_f1:.4f} | LR: {current_lr:.2e}')
        print(classification_report(all_labels, all_preds, digits=4))
        print('-' * 60)
        
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            no_improve = 0
            torch.save(model.state_dict(), 'best_model_BiLSTM_Attention.pth')
        else:
            no_improve += 1
            if no_improve >= patience:
                print(f'Early stopping at epoch {epoch+1}')
                break

In [ ]:
sampler = create_weighted_sampler(y_train)
    
train_dataset = TensorDataset(
        torch.tensor(X_train, dtype=torch.float32),
        torch.tensor(y_train, dtype=torch.long)
)
val_dataset = TensorDataset(
        torch.tensor(X_val, dtype=torch.float32),
        torch.tensor(y_val, dtype=torch.long)
)
test_dataset = TensorDataset(
        torch.tensor(X_test, dtype=torch.float32),
        torch.tensor(y_test, dtype=torch.long)
)

batch_size = 64
train_loader = DataLoader(
    train_dataset, 
    batch_size=batch_size,
    sampler=sampler,  
    num_workers=4,
    pin_memory=True
)
val_loader = DataLoader(
    val_dataset, 
    batch_size=batch_size,
    num_workers=4,
    pin_memory=True
)
test_loader = DataLoader(
    test_dataset, 
    batch_size=batch_size,
    num_workers=4,
    pin_memory=True
)


class_counts = np.bincount(y_train)
class_weights = 1. / class_counts
class_weights = class_weights / class_weights.sum() * len(class_counts)

device, model, optimizer, criterion, scheduler = initialize_model(
    X_train=X_train,
    num_classes=len(np.unique(y_train)),
    train_loader=train_loader,
    class_weights=class_weights
)

In [59]:
train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    criterion=criterion,
    device=device,
    scheduler=scheduler,
    epochs=10,
    clip_value=0.5,
    patience=5
)


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/var/folders/gg/wqlmrvl91mj28lsltv0ccxqm0000gn/T/ipykernel_10027/4196525237.py:24: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=device.type == 'cuda'):


Epoch 1/10 | Time: 41.96s
Train Loss: 0.0810 | Val Loss: 2.4137
Val F1-score: 0.6640 | LR: 1.00e-04
              precision    recall  f1-score   support

           0     0.5597    0.5325    0.5458      3012
           1     0.3041    0.4045    0.3472      3713
           2     0.8169    0.7492    0.7816     13025

    accuracy                         0.6513     19750
   macro avg     0.5602    0.5621    0.5582     19750
weighted avg     0.6813    0.6513    0.6640     19750

------------------------------------------------------------


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/var/folders/gg/wqlmrvl91mj28lsltv0ccxqm0000gn/T/ipykernel_10027/4196525237.py:24: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=device.type == 'cuda'):


Epoch 2/10 | Time: 44.88s
Train Loss: 0.0752 | Val Loss: 2.4295
Val F1-score: 0.6763 | LR: 1.00e-04
              precision    recall  f1-score   support

           0     0.5167    0.6106    0.5597      3012
           1     0.3300    0.3555    0.3423      3713
           2     0.8258    0.7729    0.7985     13025

    accuracy                         0.6697     19750
   macro avg     0.5575    0.5797    0.5668     19750
weighted avg     0.6854    0.6697    0.6763     19750

------------------------------------------------------------


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/var/folders/gg/wqlmrvl91mj28lsltv0ccxqm0000gn/T/ipykernel_10027/4196525237.py:24: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=device.type == 'cuda'):


...

Epoch 8/10 | Time: 44.47s
Train Loss: 0.0207 | Val Loss: 4.3865
Val F1-score: 0.6848 | LR: 5.00e-05
              precision    recall  f1-score   support

           0     0.5866    0.5478    0.5665      3012
           1     0.3595    0.2494    0.2945      3713
           2     0.7852    0.8657    0.8235     13025

    accuracy                         0.7014     19750
   macro avg     0.5771    0.5543    0.5615     19750
weighted avg     0.6749    0.7014    0.6848     19750

------------------------------------------------------------


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/var/folders/gg/wqlmrvl91mj28lsltv0ccxqm0000gn/T/ipykernel_10027/4196525237.py:24: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=device.type == 'cuda'):


Epoch 9/10 | Time: 43.72s
Train Loss: 0.0191 | Val Loss: 4.3729
Val F1-score: 0.6869 | LR: 2.50e-05
              precision    recall  f1-score   support

           0     0.6061    0.5395    0.5709      3012
           1     0.3492    0.3113    0.3292      3713
           2     0.7940    0.8387    0.8157     13025

    accuracy                         0.6939     19750
   macro avg     0.5831    0.5632    0.5719     19750
weighted avg     0.6817    0.6939    0.6869     19750

------------------------------------------------------------


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/var/folders/gg/wqlmrvl91mj28lsltv0ccxqm0000gn/T/ipykernel_10027/4196525237.py:24: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=device.type == 'cuda'):


Epoch 10/10 | Time: 33.94s
Train Loss: 0.0104 | Val Loss: 4.7945
Val F1-score: 0.6861 | LR: 2.50e-05
              precision    recall  f1-score   support

           0     0.6127    0.5226    0.5641      3012
           1     0.3628    0.2507    0.2965      3713
           2     0.7805    0.8758    0.8254     13025

    accuracy                         0.7044     19750
   macro avg     0.5853    0.5497    0.5620     19750
weighted avg     0.6764    0.7044    0.6861     19750

------------------------------------------------------------


In [27]:
%cd Documents/GitHub/Hates-speech-detection/notebooks/

/Users/diana/Documents/GitHub/Hates-speech-detection/notebooks


In [60]:
model.load_state_dict(torch.load('best_model_BiLSTM_Attention.pth', map_location=device))
model.eval()
    
all_preds = []
all_labels = []
    
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
    
print("\nFinal Test Results:")
print(classification_report(all_labels, all_preds, digits=4))

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)



Final Test Results:
              precision    recall  f1-score   support

           0     0.5688    0.5745    0.5716      2921
           1     0.3710    0.2476    0.2970      3780
           2     0.7991    0.8743    0.8350     13050

    accuracy                         0.7100     19751
   macro avg     0.5796    0.5654    0.5679     19751
weighted avg     0.6831    0.7100    0.6931     19751

